In [12]:
import pandas as pd
import numpy as np
import pickle

In [13]:
re = pd.read_pickle('recommend.pkl')
re

,title,categoryName,salesPoint,adult,rank,genres,mood,interest
0,자기만의 방,에세이,0,False,0,"[시대, 동화, 예술]","[자유, 행복, 위로, 열망]","[사랑, 생각, 사회, 페미니즘, 인생]"
1,오만과 편견,소설/시/희곡,0,False,0,"[시대, 동화]","[도전, 열정, 행복]","[사랑, 심리, 인간, 인생, 청춘, 관계, 페미니즘, 생각, 일상]"
2,프랑켄슈타인,소설/시/희곡,0,False,0,"[과학, 동화, 판타지, 예술]","[열망, 열정, 자유, 희망, 고민]","[심리, 동물, 인간, 관계]"
3,[큰글씨책] 원서발췌 신선전,소설/시/희곡,0,False,0,"[과학, 예술, 철학]",[],"[종교, 자연]"
4,집착,소설/시/희곡,0,False,0,"[예술, 시대, 자서전, 역사]","[고민, 열정]","[심리, 관계, 인간, 사회, 생각, 인생, 사랑]"
...,...,...,...,...,...,...,...,...
54971,뇌력혁명 - 뇌피로가 풀려야 인생이 풀린다!,자기계발,281,False,9,"[과학, 시대]","[행복, 우울, 불안, 고민, 긍정]","[사회, 일상, 인간, 인생, 청춘, 시간]"
54972,남자의 교과서 - 여자는 전혀 모르고 남자는 차마 말 못하는 것들,자기계발,281,False,9,[동화],"[슬픔, 행복, 고민, 후회]","[정치, 관계, 생각, 인생, 가족, 친구, 시간]"
54973,그리스 신전에서 인간의 길을 묻다 - 내 인생을 바꾸는 4천 년의 지혜,자기계발,281,False,8,"[예술, 시대, 판타지, 동화]","[열정, 도전, 용기, 행복]","[성공, 인간, 인생, 청춘, 일상]"
54974,성공을 부르는 긍정의 힘,자기계발,281,False,7,[],[],[성공]


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
re['genres_literal'] = re['genres'].apply(lambda x : (' ').join(x))
re['genres_literal']

0            시대 동화 예술
1               시대 동화
2        과학 동화 판타지 예술
3            과학 예술 철학
4        예술 시대 자서전 역사
             ...     
54971           과학 시대
54972              동화
54973    예술 시대 판타지 동화
54974                
54975           시대 철학
Name: genres_literal, Length: 54976, dtype: object

In [9]:
# CountVectorizer로 학습시켰더니 54976개 도서에 대한 144개 장르의 '장르 매트릭스'가 생성되었다.
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 1)) 
#min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat = count_vect.fit_transform(re['genres_literal'])
print(genre_mat.shape)

(54976, 12)


In [10]:
print(genre_mat)

  (0, 5)	1
  (0, 1)	1
  (0, 7)	1
  (1, 5)	1
  (1, 1)	1
  (2, 1)	1
  (2, 7)	1
  (2, 0)	1
  (2, 11)	1
  (3, 7)	1
  (3, 0)	1
  (3, 10)	1
  (4, 5)	1
  (4, 7)	1
  (4, 8)	1
  (4, 6)	1
  (5, 7)	1
  (5, 0)	1
  (5, 10)	1
  (6, 7)	1
  (6, 8)	1
  (6, 6)	1
  (8, 1)	1
  (9, 5)	1
  (9, 7)	1
  :	:
  (54960, 1)	1
  (54960, 10)	1
  (54961, 5)	1
  (54963, 5)	1
  (54963, 9)	1
  (54965, 5)	1
  (54965, 6)	1
  (54965, 9)	1
  (54966, 5)	1
  (54966, 1)	1
  (54966, 10)	1
  (54967, 5)	1
  (54968, 5)	1
  (54969, 5)	1
  (54969, 10)	1
  (54970, 5)	1
  (54971, 5)	1
  (54971, 0)	1
  (54972, 1)	1
  (54973, 5)	1
  (54973, 1)	1
  (54973, 7)	1
  (54973, 11)	1
  (54975, 5)	1
  (54975, 10)	1


In [11]:
# 코사인 유사도에 의해 54976개 영화 각각 유사한 영화들이 계산됨
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:5])

MemoryError: Unable to allocate 22.5 GiB for an array with shape (54976, 54976) and data type float64

In [ ]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 영화를 앞에서부터 순서대로 보여줌
# 0번째 영화의 경우 유사도 순서 : 0번, 3494번, 813번, ..., 2401 순서
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 전체를 -1칸 간격으로
print(genre_sim_sorted_ind[:5])

In [ ]:
genre_sim_sorted_ind